### Mount Google Drive to runtime

In [ ]:
from google.colab import drive
from os.path import join

# Mounting location on runtime for GDrive
ROOT = '/content/drive'

# Project workspace on GDrive
PROJECT_PATH = 'My Drive/Github'

# Mount GDrive on the runtime
drive.mount(ROOT)

# Create the full runtime project path and create a workspace at that location
WORKING_PATH = join(ROOT, PROJECT_PATH)
!mkdir "{WORKING_PATH}" 
%cd "{WORKING_PATH}"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
mkdir: cannot create directory ‘/content/drive/My Drive/Github’: File exists
/content/drive/My Drive/Github


### Install Selenium and chromedriver

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

# Add chromedriver location to path
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

     |████████████████████████████████| 911kB 4.7MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [801 B]
Get:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.c

In [ ]:
'''
@file       FlowsterWebscraper.py
@date       2020/06/09
@brief      Class to scrape attributes of interest from all topics on the Flowster Discourse forum
'''

import time
from datetime import datetime
import os

from bs4 import BeautifulSoup
from selenium import webdriver

import pandas as pd
import json


'''
@brief  Webscraper that scrapes attributes of interest from all topics on the Flowster Discourse forum
'''
class FlowsterWebscraper:
    driver = None                   # Selenium webdriver object
    topicDict = {}                  # Dictionary of all topics and their attributes
    topicDataframe = \
        pd.DataFrame(columns=[      # Pandas dataframe of all topic attributes
        'Topic Title', 
        'Category', 
        'Tags', 
        'Author', 
        'Leading Comment', 
        'Likes',
        'Views'])


    def __init__(self, webdriverPath):
        # Set up webdriver
        options = webdriver.ChromeOptions()
        options.add_argument('--ignore-certificate-errors')     # Ignore security certificates
        options.add_argument('--incognito')                     # Use Chrome in Incognito mode
        options.add_argument('--headless')                      # Run in background
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        self.driver = webdriver.Chrome( \
            executable_path = webdriverPath, \
            options = options)


    '''
    @brief      Retrieves a topic title
    @param      topicSoup   BeautifulSoup object that contains the topic page HTML
    @return     topicName   Topic name
    '''
    def get_title(self, topicSoup):
        topicName = topicSoup.find('a', class_='fancy-title').text

        # Remove leading and trailing spaces and newlines
        topicName = topicName.replace('\n', '').strip()
        return topicName


    '''
    @brief      Retrieves a topic's category and tags
    @param      topicSoup   BeautifulSoup object that contains the topic page HTML
    @return     category    Category that the topic belongs to
    @return     tags        List of topic tags
    '''
    def get_category_and_tags(self, topicSoup):    
        topicCategoryDiv = topicSoup.find('div', class_='topic-category ember-view')
        tagAnchors = topicCategoryDiv.find_all('span', class_='category-name')

        tagList = []
        for anchor in tagAnchors:
            tagList.append(anchor.text)
        
        if (len(tagList) == 1):
            category = tagList[0]
            tags = []
            return category, tags
        else:
            category = tagList[0]
            tags = tagList[1:]
            return category, tags

    
    '''
    @brief      Retrieves a topic's author and commenters
    @param      topicSoup   BeautifulSoup object that contains the topic page HTML
    @return     author      Author username
    @return     commenters  List of unique commenter usernames
    '''
    def get_authors(self, topicSoup):
        names = topicSoup.find_all("div", class_="names trigger-user-card")
        authorList = []
        for name in names:
            author = name.span.a.text
            authorList.append(author)
        return author


    '''
    @brief      Retrieves a topic's comments
    @param      topicSoup       BeautifulSoup object that contains the topic page HTML
    @return     leadingComment  Leading comment (by the author)
    @return     otherComments   List of other comments
    '''
    def get_comments(self, topicSoup):
        postStream = topicSoup.find('div', class_='post-stream')
        postDivs = postStream.find_all('div', \
            {'class':['topic-post clearfix regular','topic-post clearfix topic-owner regular']})

        comments = []
        for i in range(len(postDivs)):
            comment = postDivs[i].find('div', class_='cooked').text
            comments.append(comment)
        return comments


    '''
    @brief      Retrieves a topic's number of views
    @param      topicSoup           BeautifulSoup object that contains the topic page HTML
    @return     views.span.text     Number of views as a string
    '''
    def get_views(self, topicSoup):
        views = topicSoup.find('li', class_='secondary views')
        if views == None:
            return str(0)
        return views.span.text
        

    '''
    @brief      Retrieves a topic's number of likes
    @param      topicSoup           BeautifulSoup object that contains the topic page HTML
    @return     likes.span.text     Number of likes as a string
    '''
    def get_likes(self, topicSoup):
        likes = topicSoup.find('li', class_='secondary likes')
        if likes == None:
            return str(0)
        return likes.span.text
    
    def get_replies(self, topicSoup):
        replies = topicSoup.find('li', class_='replies')
        if replies == None:
            return str(0)
        return replies.span.text
    

    '''
    @brief      Runs the webscraper application and saves the data in both JSON and CSV files
    @param      baseURL     Link to the Flowster forum home page
    @return     None
    '''
    def runApplication(self, baseURL):
        # Open Chrome web client using Selenium and retrieve page source
        self.driver.get(baseURL)
        baseHTML = self.driver.page_source

        # Get base HTML text and generate soup object
        baseSoup = BeautifulSoup(baseHTML, 'html.parser')

        # Find all anchor objects that contain category information
        categoryAnchors = baseSoup.find_all('a', class_='category-title-link')

        # Get hyperlink references and append it to the base URL to get the category page URLs
        categoryPageURLs = []
        for i in range(len(categoryAnchors)):
            href = categoryAnchors[i]['href']
            categoryPageURLs.append(baseURL + href)

        # 1st for loop to loop through all categories
        for categoryURL in categoryPageURLs:
            # Access category webpage
            self.driver.get(categoryURL)

            # Load the entire webage by scrolling to the bottom
            lastHeight = self.driver.execute_script("return document.body.scrollHeight")
            while (True):
                # Scroll to bottom of page
                self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

                # Wait for new page segment to load
                time.sleep(0.5)

                # Calculate new scroll height and compare with last scroll height
                newHeight = self.driver.execute_script("return document.body.scrollHeight")
                if newHeight == lastHeight:
                    break
                lastHeight = newHeight

            # Generate category soup object
            categoryHTML = self.driver.page_source
            categorySoup = BeautifulSoup(categoryHTML, 'html.parser')

            # Find all anchor objects that contain topic information
            topicAnchors = categorySoup.find_all('a', class_='title raw-link raw-topic-link')

            # Get hyperlink references and append it to the base URL to get the topic page URLs
            topicPageURLs = []
            for i in range(len(topicAnchors)):
                href = topicAnchors[i]['href']
                topicPageURLs.append(baseURL + href)


            # 2nd for loop to loop through all topics in a category
            for topicURL in topicPageURLs:
                # Get topic HTML text and generate topic soup object
                self.driver.get(topicURL)
                topicHTML = self.driver.page_source
                topicSoup = BeautifulSoup(topicHTML, 'html.parser')

                # Scape all topic attributes of interest
                topicTitle = self.get_title(topicSoup)
                category, tags = self.get_category_and_tags(topicSoup)
                author= self.get_authors(topicSoup)
                leadingComment= self.get_comments(topicSoup)
                numLikes = self.get_likes(topicSoup)
                numViews = self.get_views(topicSoup)
                replies = self.get_replies(topicSoup)
                
                counter = 1
                for post in leadingComment:

                    # Create attribute dictionary for topic
                    attributeDict = {
                        'Topic Title'       :   topicTitle + str(counter),
                        'Category'          :   category,
                        'Tags'              :   tags,
                        'Author'            :   author,
                        'Leading Comment'   :   post,
                        'Likes'             :   numLikes,
                        'Replies'           :   replies,
                        'Views'             :   numViews}

                    # Add the new entry to the topic dictionary and Pandas dataframe
                    self.topicDict[topicTitle + str(counter)] = attributeDict
                    self.topicDataframe = self.topicDataframe.append(attributeDict, ignore_index=True)
                    counter += 1
                
        # Get unique timestamp of the webscraping
        timeStamp = datetime.now().strftime('%Y%m%d%H%M%S')

        # Save data in JSON and CSV files and store in the save folder as this program
        jsonFilename = 'Flowster_Topic_Attributes_' + timeStamp + '.json'
        csvFilename = 'Flowster_Topic_Attributes_' + timeStamp + '.csv'

        #jsonFileFullPath = os.path.join(os.path.dirname(os.path.realpath("__file__")), jsonFilename)
        #csvFileFullPath = os.path.join(os.path.dirname(os.path.realpath("__file__")), csvFilename)

        jsonFileFullPath = os.path.join(WORKING_PATH, jsonFilename)
        csvFileFullPath = os.path.join(WORKING_PATH, csvFilename)

        with open(jsonFileFullPath, 'w') as f:
            json.dump(self.topicDict, f)

        self.topicDataframe.to_csv(csvFileFullPath)



if __name__=='__main__':
    # Local path to webdriver
    webdriverPath = "chromedriver"

    # Flowster forum base URL
    baseURL = 'https://forum.flowster.app'

    # Create FLowster webscraping object
    flowsterWebscraper = FlowsterWebscraper(webdriverPath)

    # Run webscraping and save data
    flowsterWebscraper.runApplication(baseURL)
